In [139]:
import petl as etl
table = etl.fromcsv('leben_base.csv')
# etl.fieldnames(table)

In [140]:
ignored_fileds = ['¿Por qué?', '¿Qué te gustaría que te ofreciera Wolox que hoy no te ofrece?', '¿Qué es lo que más te gusta de trabajar en Wolox?', '¿Qué es lo que menos te gusta de trabajar en Wolox?', '¿Cómo ves a Wolox de acá a 5 años?', 'Describí a Wolox en máximo 3 palabras', 'Email Address']


fieldlabels ={
 'Timestamp': 'TIMESTAMP',
 '¿En qué país trabajaste la mayor parte del 2018?': 'PAIS',
 '¿En qué oficina trabajás actualmente o trabajaste mayoritariamente cuando estuviste en Argentina ?': 'SEDE',
 '¿Trabajaste antes en las oficinas de Wolox Argentina?': 'RELOC',
 '¿Considerás tu relocación como una experiencia positiva?': 'EXP_RELOC',
 'Trabajo en el área:': 'AREA',
 'Llevo trabajando en Wolox:': 'ANTIGUEDAD',
 'Creo que mi día a día en Wolox...': 'SATISFACCION',
 'Considero que las personas con las que trabajo son:': 'PARES',
 'Elijo trabajar en Wolox porque:': 'ELIJO',
 'Creo que Wolox la rompe en:': 'BIEN',
 'Creo que Wolox debería mejorar en:': 'MAL',
 'Trabajar en Wolox:': 'ME_GUSTA',
 '¿Le recomendarías a alguien trabajar en Wolox?': 'RECOMIENDA' }

In [141]:
table = etl.cutout(table, *ignored_fileds)

# TODO: Remover la doble columna
table = etl.cutout(table, '¿Por qué?')

In [142]:
# etl.fieldnames(table)

In [143]:
table = etl.rename(table, fieldlabels)
# etl.fieldnames(table)

In [144]:
numeric_values = {
    'PAIS': {
        'Argentina': 0,
        'Chile': 1,
        'Colombia': 2,
        'México': 3,
        'Estados Unidos': 4,
        '': 0
    },
    'SEDE': {
        'Güemes': 0,
        'Azurduy': 0.5
    },
    'RELOC': {
        'No, siempre estuve en el mismo país': 0,
        'Sí, trabajé en Argentina antes': 1,
        '': 0
    },
    'EXP_RELOC': {
        'Sí': 10,
        'Más o menos': 5,
        'No': 0,
        '': 0
    },
    'AREA': {
        'Desarrollo': 0,
        'Management': 1,
        'QA': 2, 
        'People Care': 3,
        'X Lab': 4,
        'Comercial': 5,
        'Legales': 6,
        'Calidad': 7,
        'Administración y Finanzas': 8,
        'Marketing': 9,
        '': 0
    },
    'ANTIGUEDAD': {
        'Menos de 6 meses': 0.25,
        'De 6 meses a 1 año': 0.75,
        'Entre 1 y 2 años': 1.5,
        'Entre 2 y 3 años': 2.5,
        'Entre 3 y 4 años': 3.5,
        'Entre 4 y 5 años': 4.5,
        'Más de 5 años': 6,
        '': 1
    },
    'SATISFACCION': {
        '': 7
    },
    'PARES': {
        'Personas con las que me gusta compartir el día a día, pero sólo en el ámbito laboral': 10,
        'Personas con las que comparto intereses y motivaciones dentro y fuera del trabajo': 6,
        'Personas con las que no tengo mucho en común': 3,
        'Personas con las que no siento comodidad': 0,
        '': 5
    },
    'ME_GUSTA': {
        'Me gusta': 10,
        'Me es indiferente': 5,
        'No me gusta': 0,
        '': 5
    },
    'RECOMIENDA': {
        'Sí, seguro': 10,
        'Tal vez': 5,
        'No, de ninguna manera': 0,
        '': 5
        
    }     
}

In [145]:
table = etl.convert(table, numeric_values)


In [146]:
# NULL VALUES THAT DEPEND ON OTHER VALUES
table = etl.convert(table, 'SEDE', lambda v, row: row.PAIS, pass_row=True)

In [147]:

etl.tocsv(table, 'example.csv')

elijolabels = {
    'Me gusta el entorno de trabajo': 'ELIJO_ENTORNO', 
    'Me gusta lo que hago': 'ELIJO_LO_QUE_HAGO', 
    'Aprendo': 'ELIJO_APRENDO',
    'Le encuentro gran potencial para mi carrera': 'ELIJO_CARRERA',
    'Estoy seguro/a de que Wolox la va a romper': 'ELIJO_VA_A_ROMPER',
    'Me siento cómodo/a': 'ELIJO_COMODO',
    'Me pagan bien': 'ELIJO_PAGAN_BIEN'}

bienlabels = {
    'La gente que la compone': 'BIEN_GENTE', 
    'La cultura': 'BIEN_CULTURA', 
    'Las oficinas': 'BIEN_OFICINA', 
    'Las capacitaciones': 'BIEN_CAPACITACIONES', 
    'El equipamiento (hardware)': 'BIEN_HARDWARE', 
    'Los proyectos': 'BIEN_PROYECTOS',
    'Los sueldos': 'BIEN_SUELDOS'
}

mallabels = {
    'La gente que la compone': 'MAL_GENTE', 
    'La cultura': 'MAL_CULTURA', 
    'Las oficinas': 'MAL_OFICINA', 
    'Las capacitaciones': 'MAL_CAPACITACIONES', 
    'El equipamiento (hardware)': 'MAL_HARDWARE', 
    'Los proyectos': 'MAL_PROYECTOS',
    'Los sueldos': 'MAL_SUELDOS'
}

for key, value in elijolabels.items():
  table = etl.addfield(table, value, lambda rec, key=key: 1 if (key in rec['ELIJO']) else 0)

for key, value in bienlabels.items():
  table = etl.addfield(table, value, lambda rec, key=key: 1 if (key in rec['BIEN']) else 0)

for key, value in mallabels.items():
  table = etl.addfield(table, value, lambda rec, key=key: 1 if (key in rec['MAL']) else 0)

table = etl.cutout(table, 'ELIJO')
table = etl.cutout(table, 'BIEN')
table = etl.cutout(table, 'MAL')


In [148]:
etl.tocsv(table, 'leben_num.csv')